In [18]:
import lancedb
import os
import getpass
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import LanceDB



ModuleNotFoundError: No module named 'langchain_openai'

In [2]:
uri = "../db/lance.db"
db = lancedb.connect(uri)

In [3]:
tbl = db.create_table("my_table",
                data=[{"vector": [3.1, 4.1], "item": "foo", "price": 10.0},
                      {"vector": [5.9, 26.5], "item": "bar", "price": 20.0}])


In [4]:
import pandas as pd
df = pd.DataFrame([{"vector": [3.1, 4.1], "item": "foo", "price": 10.0},
                   {"vector": [5.9, 26.5], "item": "bar", "price": 20.0}])
tbl = db.create_table("table_from_df", data=df)

In [5]:
import pyarrow as pa
schema = pa.schema([pa.field("vector", pa.list_(pa.float32(), list_size=2))])
tbl = db.create_table("empty_table", schema=schema)


In [6]:
tbl = db.open_table("my_table")


In [7]:
print(db.table_names())


['empty_table', 'my_table', 'table_from_df']


In [12]:
# Option 1: Add a list of dicts to a table
data = [{"vector": [1.3, 1.4], "item": "fizz", "price": 100.0},
        {"vector": [9.5, 56.2], "item": "buzz", "price": 200.0}]
tbl.add(data)

# Option 2: Add a pandas DataFrame to a table
df = pd.DataFrame(data)
tbl.add(data)


In [20]:
# How to search for (approximate) nearest neighbors

tbl.search([100, 100]).limit(2).to_pandas()


OSError: Io error: Execution error: External error: Execution error: LanceError(IO): Execution error: Failed to compute distances: LanceError(IO): Execution error: Not found: home/james/Dropbox/GIT/CS6795-Cognitive_Science/CS6795-Cognitive_Science_Code/pdf_chat-dev/db/lance.db/my_table.lance/data/d7c5c438-4445-4ba2-85d1-5eb95b120353.lance, /home/runner/work/lance/lance/rust/lance-io/src/local.rs:93:31, /home/runner/work/lance/lance/rust/lance/src/dataset/scanner.rs:1333:31, /home/runner/work/lance/lance/rust/lance/src/io/exec/take.rs:72:42

In [14]:
tbl.create_index()


OSError: LanceError(Index): KMeans: can not train 256 centroids with 10 vectors, choose a smaller K (< 10) instead, /home/runner/work/lance/lance/rust/lance-index/src/vector/kmeans.rs:45:21

In [15]:
tbl.delete('item = "fizz"')


In [19]:
db.drop_table("my_table")


FileNotFoundError: [Errno 2] Cannot delete directory '/home/james/Dropbox/GIT/CS6795-Cognitive_Science/CS6795-Cognitive_Science_Code/pdf_chat-dev/src/../db/lance.db/my_table.lance': Cannot get information for path '/home/james/Dropbox/GIT/CS6795-Cognitive_Science/CS6795-Cognitive_Science_Code/pdf_chat-dev/src/../db/lance.db/my_table.lance'. Detail: [errno 2] No such file or directory

In [ ]:
# Create embeddings
embeddings = OpenAIEmbeddings()


In [21]:


db = lancedb.connect("../db/lancedb")

table = db.create_table(
    "my_table",
    data=[
        {
            "vector": embeddings.embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader('../../../state_of_the_union.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = LanceDB.from_documents(documents, OpenAIEmbeddings())



NameError: name 'embeddings' is not defined

In [ ]:
# Seach
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[0].page_content)

